Estimators 封装在下列actions当中：
<ul type='disc'>
    <li>training</li>
    <li>evaluation</li>
    <li>prediction</li>
    <li>export for serving</li>
</ul>
<font size="2" color="grey"> 所有的Estimators都基于tf.estimator.Estimators 类(tf.contrib.learn.Estimator)已经被废弃，不应该继续使用</font>   

<b> Estimator 优势：</b>

<ul>
    <li>使用Estimator_based模型，不需要在不同的平台上修改代码</li>
    <li>Estimators simplify sharing implementations between model developers</li>
    <li>You can develop a state of the art model with high-level intuitive code, In short, it is generally much easier to create models with Estimators than with the low-level TensorFlow APIs.</li>
    <li>Estimators 简历在tf.layers上面，可以简单的被自定义</li>
    <li>Estimators可以自动的创建Graph</li>
    <li>Estimators可以提供safe distributed training loop that controls how and when to:</li>
    <ul>
    <li>构建图形</li>
    <li>初始化变量</li>
    <li> start queues</li>
    <li>处理已成</li>
    <li>create checkpoint files and recover from failures</li>
    <li>为Tensorboard保存summaries</li>
    </ul>
    </ul>  
当写一个具有Estimators的应用是，you must separate the data input pipeline from the model.这样做可便于实验数据的替换。


# Pre-made Estimators
pre-made Estimators可以创建和管理Graph和Session对象。   
Furthermore, pre-made Estimators let you experiment with different model architectures by making only minimal code changes. DNNClassifier, for example, is a pre-made Estimator class that trains classification models through dense, feed-forward neural networks.

<b>tf.estimator.LinearClassifier:</b> Constructs a linear classification model.
<b>tf.estimator.LinearRegressor:</b> Constructs a linear regression model.  
<b>tf.estimator.DNNClassifier:</b> Construct a neural network classification model.  
<b>tf.estimator.DNNRegressor:</b> Construct a neural network regression model.  
<b>tf.estimator.DNNLinearCombinedClassifier:</b> Construct a neural network and linear combined classification model.  
<b>tf.estimator.DNNLinearCombinedRegressor:</b> Construct a neural network and linear combined regression model.  

### Structure of a pre-made Estimators program
1.<b> 写一个或者多个dataset导入函数。</b>  
input_fn
<p> 每个dataset导入函数返回两个对象：
    <ul>
        <li> 字典：keys：特征名称，values：Tensor(or sparse Tensros)对应相应的特征。</li>
        <li> Tensor: 包含一个或者多个标签</li>
    </ul>
</p>
<p>
    input function(input_fn)的主题包含了input data预处理过程，必须要返回两个值包含处理过的feature和label(用于传递给模型)：（函数的skeleton见上面)my_input_fn):  
    <ul>
        <li><b>feature_cols: </b>dict: 包含key/value成对对应到feature columns names和包含相应特征的Tensor(Sparse Tensor)</li>
        <li><b>labels: </b>a Tensor包含目标值(label)，即模型的预测值</li>
    </ul>
</p>
<p>
    如果feature/label data使用python array存储在pandas dataframe或者numpy array中，可以用以下方法来构建input_fn:详细例子如下
</p>
<p>
    对于sparse,categorical data(大部分值为0),那么input_fn可以改为填充一个SparseTensor，可以通过三个参数实例化：
    <ul>
        <li><b>dense_shape: </b>  
            the shape of tensor.list类型，每一个元素代表当前的维度，e.g.dense_shape=[3,6],[2,3,4],[9]分别代表二维，三维，一维的tensor
        </li>
        <li><b>indices: </b>  
            知名非零值出现的位置，list类型。index以0开始，即[0,0]表示第一行第一列，indices=[[1,3],[2,4]]代表[1,3],[2,4]的index处不为零
        </li>
        <li><b>values: </b>  
            一维tensor，第i个值表示indices中的第i个元素填充的值
        </li>
    </ul>
</p>

In [5]:
import tensorflow as tf
#e.g.: 
def input_fn(dataset):
    ...# manipulate dataset, extracting feature names and the label
    return feature_dict,label

def my_input_fn():
    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels

import numpy as np
# numpy input_fn.
my_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(x_data)},
    y=np.array(y_data),
    ...)

import pandas as pd
# pandas input_fn.
my_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=pd.DataFrame({"x": x_data}),
    y=pd.Series(y_data),
    ...)



In [22]:
sparse_tensor = tf.SparseTensor(indices=[[0,1], [2,4]],
                                values=[6, 0.5],
                                dense_shape=[3, 5])
with tf.Session() as sess:
    print(sess.run(sparse_tensor).dense_shape)

[3 5]


2.<b> 定义特征columns。</b>每一个tf.feature_column定义(identifies)一个特征名字，他的类型和任何的预处理（input pre-processing)  
<p>这个例子中创建了三个特征columns包含integer或者是float-point数据
</p>


In [8]:
# Define three numeric feature columns
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column('median_edcation',
                                                     normalizer_fn='lambda x:x-global_education_mean')

3.<b> 实例化相关的pre-made Estimator。</b>
<p>这个例子中实例化pre-made Estimator为LinearClassifier
</p>

In [10]:
# Instantiate an estimator, passing the feature columns.
estimator = tf.estimator.Estimator.LinearClassifier(
    feature_columns=[population,crime_rate,median_education],)

4.<b> Call a training, evaluation, or inference method。</b>
<p>所有的Estimator都提供一个train模型
</p>

In [ ]:
# my_training_set is the function created in Step 1
estimator.train(input_fn=my_training_set,steps=2000)

## Benefits of pre-made Estimators
Pre-made Estimators encode best practices, providing the following benefits:  
<ul>
    <li>Best practices for determining where different parts of the computational graph should run, implementing strategies on a single machine or on a cluster.  
    （决定计算图形哪些不同的地方应当运行，在一个机器或者一个cluster上实行决策的最好的实现）
    </li>
    <li>Best practices for event (summary) writing and universally useful summaries.（event写定和普遍有用的总结的实现）</li>
</ul>
如果不适用pre-made Estimators,那么需要自行实践preceding features
    

In [ ]:
## pre-made Estimator 实例：DNNclassifier，实行dense，feed-forward NN

categorical_feature_a = categorical_column_with_hash_bucket(...)
categorical_feature_b = categorical_column_with_hash_bucket(...)

categorical_feature_a_emb = embedding_column(
    categorical_column=categorical_feature_a, ...)
categorical_feature_b_emb = embedding_column(
    categorical_column=categorical_feature_b, ...)

estimator = DNNClassifier(
    feature_columns=[categorical_feature_a_emb, categorical_feature_b_emb],
    hidden_units=[1024, 512, 256])

# Or estimator using the ProximalAdagradOptimizer optimizer with
# regularization.
estimator = DNNClassifier(
    feature_columns=[categorical_feature_a_emb, categorical_feature_b_emb],
    hidden_units=[1024, 512, 256],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

# Input builders
def input_fn_train: # returns x, y
    pass
    #e.g.  
     return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y = pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)
estimator.train(input_fn=input_fn_train, steps=100)

def input_fn_eval: # returns x, y
    pass
metrics = estimator.evaluate(input_fn=input_fn_eval, steps=10)
def input_fn_predict: # returns x, None
    pass
predictions = estimator.predict(input_fn=input_fn_predict)

# Custom Estimators
每一个Estimator的核心（不管是pre-made还是custom）都是model function，
是一个建立training, evaluation, 和prediction的方法。  
当用 pre-made Estimator的时候，别人已经implemented the model function。
在custom Estimator的时候，必须要自己写一个方程  
步骤：
<ul>
    <li>Instantiate an Estimator</li>
    <li>Construct a custom model function</li>
    <li>Configure a neural network using tf.feature_column and tf.layers</li>
    <li>Choose an appropriate loss function from tf.losses</li>
    <li>Define a training op for your model</li>
    <li>Generate and return predictions</li>
</ul>

## Recommended workflow
建议流程：
1. 假设一个合适的pre-made Estimator存在，用这个来创建你的第一个模型，并且用这个结果来建立baseline。
2. 创建和检测（build and test）你所有的pipeline，包含在这个pre-made Estimator数据的完整性和可靠性(integrity and reliability)
3. 如果有可供选择用来替换的pre-made Estimator是存在的，运行这个pre-made Estimator创建了的最好结果。
4. 可能的情况下，可以通过你自己自定义custom Estimator来提升你的模型。

nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)
<ul>
    <li><b>model_fn:</b> A function object that contains all the aforementioned logic to support training, evaluation, and prediction. You are responsible for implementing that functionality. The next section, Constructing the model_fn covers creating a model function in detail.</li>

<li><b>params:</b> An optional dict of hyperparameters (e.g., learning rate, dropout) that will be passed into the model_fn.</li>
</ul>

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

# Import urllib
from six.moves import urllib

import numpy as np
import tensorflow as tf

FLAGS = None
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def maybe_download(train_data, test_data, predict_data):
    """Maybe downloads training data and returns train and test file names."""
    if train_data:
        train_file_name = train_data
    else:
        train_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_train.csv",
            train_file.name)
        train_file_name = train_file.name
        train_file.close()
        print("Training data is downloaded to %s" % train_file_name)

    if test_data:
        test_file_name = test_data
    else:
        test_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_test.csv", test_file.name)
        test_file_name = test_file.name
        test_file.close()
        print("Test data is downloaded to %s" % test_file_name)

    if predict_data:
        predict_file_name = predict_data
    else:
        predict_file = tempfile.NamedTemporaryFile(delete=False)
        urllib.request.urlretrieve(
            "http://download.tensorflow.org/data/abalone_predict.csv",
            predict_file.name)
        predict_file_name = predict_file.name
        predict_file.close()
        print("Prediction data is downloaded to %s" % predict_file_name)

    return train_file_name, test_file_name, predict_file_name

In [ ]:
# instantiating an estimator
# pre-made estimator
my_nn = tf.estimator.DNNClassifier(feature_columns=[age, height, weight],
                                   hidden_units=[10, 10, 10],
                                   activation_fn=tf.nn.relu,
                                   dropout=0.2,
                                   n_classes=3,
                                   optimizer="Adam")

In [ ]:
# custom estimator
nn = tf.estimator.Estimator(model_dn=model_fn,params=model_params)# 两个参数
def model_fn(features, labels, mode, params):
   # Logic to do the following:
   # 1. Configure the model via TensorFlow operations
   # 2. Define the loss function for training/evaluation
   # 3. Define the training operation/optimizer
   # 4. Generate predictions
   # 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
    return EstimatorSpec(mode, predictions, loss, train_op, eval_metric_ops)

def my_input_fn():

    # Preprocess your data here...

    # ...then return 1) a mapping of feature columns to Tensors with
    # the corresponding feature data, and 2) a Tensor containing labels
    return feature_cols, labels

model_fn接收三个参数：
<ul>
    <li><b>feature: </b> dict：包含特征，通过input_fn传递给model</li>
    <li><b>labels: </b>Tensor：包含labels, 通过input_fn传递给model</li>
    <li><b>mode: </b>tf.estimator.ModeKeys的string值代表会工作一下那个model_fn:</li>
<p>
    input_fn见最上面的说明
</p>
    <ul>
        <li> tf.estimator.ModeKeys.TRAIN：model_fn执行training mode，通过train()调用</li>
        <li> tf.estimator.ModeKeys.EVAL: model_fn执行evaluation mode,通过evaluate()调用</li>
        <li> tf.estimator.ModeKeys.PREDICT: model_fn执行predict mode，通过predict()调用</li>
    </ul>
</ul>
model_fn 也可以接收params参数（包含一个hyperparameters的字典），用来训练  
函数的主体执行以下功能：
<ul>
    <li>配置模型:集模型的结构</li>
    <li>定义损失函数</li>
    <li>定义训练的操作，和optimizer的算法来让损失函数变小</li>
</ul>
<b>return:</b>  
返回tf.estimator.EstimatorSpec 对象，包含以下值：  
__new__(
    cls,  
    mode,  
    predictions=None,  
    loss=None,  
    train_op=None,  
    eval_metric_ops=None,  
    export_outputs=None,  
    training_chief_hooks=None,  
    training_hooks=None,  
    scaffold=None,  
    evaluation_hooks=None  
)
<ul>
    <li><b>mode(required) </b>这个model运行的mode，一般而言会在model_fn中返回mode参数</li>
    <ul>
        <li>For <b>mode == ModeKeys.TRAIN: </b>required fields are loss and train_op.</li>
        <li>For <b>mode == ModeKeys.EVAL: </b>required field is loss.</li>
        <li>For <b>mode == ModeKeys.PREDICT: </b>required fields are predictions.</li>
    </ul>
    <li><b>predictions(required in PREDICT model) </b>dict：将选择的名字于model预测值得Tensor匹配 
        e.g. python predictions = {"results":tensor_of_predictions}  
        PREDICT mode,在EstimatorSpec中得dict会通过predict()返回
    </li>
    <li><b>loss(required in EVAL and TRAIN mode) </b>A Tensor 包含一个标量损失值</li>
    <li><b> train_op (required only in TRAIN mode) </b>An Op that runs one step of training.</li>
    <li><b> eval_metric_ops (optional) </b>dict: name/value匹配特定得metrics可以被模型在EVAL模型中计算，name是选择得metric，value是计算结果，tf.metrics模块提供一些普遍的已经被定义好得函数  
        e.g. 一下的评估中包含一个‘accuracy'的metric使用tf.metrics.accuracy计算：  
      python eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels,predictions)}  
        如果不用这个参数，那么在evaluation的时候只有loss会被计算
    </li>        


### Configuring a neural network with tf.feature_column and tf.layers
<b>model construction +predictions</b>  
neural network构建包含三个部分：input layer, hidden layers和output layer  

input layer：包含一些列的nodes接收feature data(通过features参数传递到model_fn中)当features包含n-dimension Tensor(所有的特征数据),那么这层就作为input layer。  
features： dict of feature_columns，通过input function传递到model中，可以通过tf.feature_column.input_layer函数将其转换成input_layer Tensor。  
e.g. : input_layer = tf.feature_column.input_layer(features=features,feature_columns=[age, height, weight])  
input_layer()包含两个required参数：
<ul>
    <li><b>features </b>a mapping 从字符串keys到包含相对应的feature data Tensor。这就是传递到model_fn中的feature参数</li> 
    <li><b>feature_columns </b>a list 包含所有的模型中的FeatureColumns，例如以上例子中的age, height, weight</li>
</ul>
<p>
    input layer通过activation function(对前一层输出的数据进行非线性转换)和hidden layers连接，hidden layer的最后一层连接到输出层，model的最后一层，tf.layers提供tf.layers.dense方程来构建全连接层。activation通过activation参数进行控制。activation其中一些选择有：
    <ul>
        <li><b>tf.nn.relu </b>e.g. 创建layer units nodes全连接到前一层的input_layer用relu：  
            python hidden_layer= tf.layers.dense(inputs=input_layer,units=10,activation=tf.nn.relu)
        </li>
        <li><b>tf.nn.relu6 </b>计算方法：min(max(features, 0), 6)  
            e.g. python second_hidden_layer = tf.layers.dense( inputs=hidden_layer, units=20, activation=tf.nn.relu)
        </li>
        <li><b>None </b> 不连接activate function，只有linear transformation  
            python output_layer = tf.layers.dense( inputs=second_hidden_layer, units=3, activation=None)
        </li>
    </ul>
其他可能选择：  
output_layer = tf.layers.dense(inputs=second_hidden_layer,
                               units=10,
                               activation_fn=tf.sigmoid)  
上面的code船舰了一个output_layer，通过一个sigmoid激活函数全连接到second_hidden_layer
</p>

### Defining loss for the model
model_fn返回的EstimatorSpec必须要包含loss：a Tensor(表示loss值)  
tf.losses模块提供convenience functions计算不同metrics中的loss：
<ul>
    <li><b>absolute_difference(labels, predictions) </b>计算loss用absolute-difference方程计算(L1 loss)</li>
    <li><b>log_loss(labels, predictions) </b>计算loss用logistic loss formula计算(一般用logistic regression)</li>
    <li><b>mean_squared_error(labels, predictions) </b>(MSE;用L2 loss)</li>
</ul>
以下例子中的model_fn用mean_squared_error()计算  
<p>
    补充的用来evaluation的metrics加到eval_metric_ops字典当中，以下code定义rmse metric，用来在model predictions时计算root mean sqaured error。labels tensor需要转换到float64来匹配predictions tensor的数据类型（包含实数）：  
    eval_metric_ops = {"rmse":tf.metrics.root_mean_squared_error(  
         tf.cast(lables,tf.float64),predictions))

### Defining the training op for the model
training op定义优化算法。一般而言，在训练的时候，目标是最小化loss。一个简单的来创建training op的方法是实例化tf.train.Optimizer的子类并且调用它的minimize方法
<p>
    以下用例当中learning_rate传送到function的param参数当中，使用gradient descent optimizer。对于global_step，采用convenience function tf.train.get_global_step来生成一个整数变量：  
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])  
    training_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    </p>

In [ ]:
## 完整的model_fn
def model_fn(features,labels,mode,params):
    """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features["x"]) with relu activation
    first_hidden_layer = tf.layer.dense(feature['x'],10,activation=tf.nn.relu)
    
    # Connect the second hidden layer to first hidden layer with relu
    second_hidden_layer = tf.layers.dense(first_hidden_layer,10,activation=tf.nn.relu)
    
    # Connect the output layer to second hidden layer (no activation fn)
    output_layer = tf.layers.dense(second_hidden_layer,1)
    
    # Reshape output layer to 1-dim Tensor to return predictions
    predictions = tf.reshape(output_layer,[-1])
    predictions_dict = {"age":predictions}
    
    # Calculate loss using mean squared error
    loss = tf.losses.mean_squared_error(labels, predictions)
    
    # calculate root mean squared error as additional eval metric
    eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(
        tf.cast(labels, tf.float64), predictions)
    }
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=param["learning_rate"])
    train_op = optimizer.minimize(loss=loss,global_step=tf.train_get_global_step())
    
    # Provide an estimator spec for `ModeKeys.EVAL` and `ModeKeys.TRAIN` modes.
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions = predictions
        loss=loss,
        train_op=train_op
        eval_metric_ops=eval_metric_ops
    )

In [15]:
def main(unused_argv):
  # Load datasets
    abalone_train, abalone_test, abalone_predict = maybe_download(
    FLAGS.train_data, FLAGS.test_data, FLAGS.predict_data)

  # Training examples
    training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_train, target_dtype=np.int, features_dtype=np.float64)

  # Test examples
    test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_test, target_dtype=np.int, features_dtype=np.float64)

  # Set of 7 examples for which to predict abalone ages
    prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
      filename=abalone_predict, target_dtype=np.int, features_dtype=np.float64)
    
    # Set model params
    model_params = {"learning_rate": LEARNING_RATE}
    
    # Instantiate Estimator
    nn = tf.estimator.Estimator(model_fn=model_fn,params=model_params)
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":np.array(training_set.data)},
        y=np.array(training_set.target),
        num_epochs=None,
        shuffle=True)
    
    # train
    nn.train(input_fn=train_input_fn,steps=5000)
    
    # score_accuracy
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x":np.array(test_set.data)},
        y=np.array(test_set.target),
        num_epochs=1,
        shuffle=False)
    
    ev = nn.evaluate(input_fn=test_input_fn)
    print("Loss: %s" % ev["loss"])
    print("Root Mean Squared Error: %s" % ev["rmse"])
    
    # print out predictions
    predict_input_fn=tf.estimator.inputs.numpy_input_fn(
        x={"x":prediction_set.data},
        num_epochs=1,
        shuffle=False)
    predictions = nn.predict(input_fn=predict_input_fn)
    for i, p in enumerate(predictions):
        print("Prediction %s: %s" % (i + 1, p["ages"]))
    

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.register("type", "bool", lambda v: v.lower() == "true")
    parser.add_argument(
      "--train_data", type=str, default="", help="Path to the training data.")
    parser.add_argument(
      "--test_data", type=str, default="", help="Path to the test data.")
    parser.add_argument(
      "--predict_data",
      type=str,
      default="",
      help="Path to the prediction data.")
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)


## Creating Custom Estimators
以下文档介绍custom Estimators。这个文档展示了怎么样创建一个自定义的Estimator类似于pre-made Estimator

### Pre-made vs custom
在如下的图形中显示，pre-made Estimator是__tf.estimator.Estimator__基础类，自定义的Estimator是tf.estimator.Estimator的实例。
![Titile](../image/estimator_types.png)
Pre-made Estimator是完全已经弄好的。但是有时候，你需要对Estimator的行为有更多的控制，这就是需要自定义custom Estimator的时候。 你可以创建一个自定义的Estimator来做任何事。如果你想hidden layers用一种不寻常的方式连接，那么使用一个custom Estimator。通常情况下，如果你想要一个Estimator为一个特定的问题优化，也可以写一个自定义的Estimator。

一个模型函数(model_fn)实现ML算法。pre-made Estimator和自定义的Estimator的区别是:
* pre-made Estimator, 别人已经定义好了模型函数
* 自定义 Estimator,你必须自己写一个模型函数。

你的model function可以实现很多的算法，定义所有的hidden layers和测量方式(metrics)。 类似于输入方程，所有的模型函数必须要接受一个标准的输入参数，返回一组标准的输出值。就像输入方程可以利用Dataset API,模型函数也可以利用 Layers API和Metrics API。

让我们查看怎么样使用自定义的Estimator来解决Iris problem。如下显示了我们想要实现的Iris模型：
![Title](../image/full_network.png)

### Write an Input function
和re-made Estimator inplementation一样，custom Estimator也使用相同的输入函数：

这个输入方程创建了一个输入管道(input pipeline)产生batches of (features,labels)对，这些features是字典的数据结构。

In [ ]:
def trian_input_fn(fetures,labels,batch_size):
    """An input function for training"""
    # Convert the inputs to dataset
    dataset = tf.train.Dataset.from_tensor_slice(dict(features),labels)
    
    # Shuffle,repeat, and batch the examples
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    # Return the read end of the pipeline
    return dataset.make_one_shot_iterator().get_next()

### Create feature columns
与之前描述的一样，必须要定义模型的feature columns来指定模型怎么样使用这些feature。在pre-made Estimator和custom Estimator，feature column的定义方式都是一样的。

下民的代码为每一个输入的特征创建了一个简单的numeric_column，于是这输入特征的值可以直接作为模型的输入：

In [ ]:
# Feature columns describe how to use the input
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

### Write a model function 
模型函数会用下面的签名：
> def my_model_fn(  
     features, # This is batch_features from input_fn  
     labels,   # This is batch_labels from input_fn  
     mode,     # An instance of tf.estimator.ModeKeys  
     params):  # Additional configuration
    
前两个参数是从输入方程返回的batches of特征和labels，也就是说，features和labels是模型会使用的数据的句柄(handles)。__mode__ 参数预示这这个caller是在请求training，predicting或者evaluation。

这个caller将__params__传递到Estimator的构造器。任何传递给构造器的__params__都会一次传递给__model_fn__。在[custom_estimator.py](https://github.com/tensorflow/models/blob/master/samples/core/get_started/custom_estimator.py)当中，接下来的行创造了一个构造器，设定配置这个模型的参数。这个配置的步骤与之前在DNNClassifier一致。

要实现一个镜店的模型函数，你必须要做以下的事情：
* 定义模型
* 确定下面三个mode的额外的计算：
    * predict
    * Evaluate
    * Train

### Define the model
在基本的DNN模型当中，必须要定义以下三个部分:
* 一层输入层(input layer)
* 一层或者多层隐藏层(hidden layers)
* 一层输出层(output layer)

#### Define the input layer
在__model_fn__中的第一行，调用__tf.feature_column.input_layer__来将特征字典和__feature_columns__转换到模型的输入，代码如下：  
示例当中的代码实现了通过feature columns定义的转换，创建了模型的输入层。
![Title](../image/input_layer.png)

In [ ]:
# Use `input_layer` to apply the feature columns,详细说明见上方
net = tf.feature_column.input_layer(features,params['feature_columns'])

#### Hidden Layers
Layers API提供了定义所有类型hidden layers的函数，包括了卷积，池化和dropout层。在Iris,我们单纯调用__tf.layers.dense__创建hidden layers，维度有params['hidden_layer']控制。在dense layer，米一个node都与前一层的每个node相连接，相关代码如下。
* __tf.layers.dense（）__ 中的__units__参数定义了给定层的输出的neurons个数。
* __activation__ 参数定义了激活函数

变量__net__表示是网络的当前顶层。在第一次迭代，__net__表明输入层(input layer)。在每一次循环迭代中，__tf.layers.新的一层，利用__net__变量来将前一层的输出作为当前层的输入。
    
在创建了两层hidden layers之后，构建的网络如下所示。为了简化，下图中并未显示每层全部的units。
![Title](add_hidden_layer.png)
注意到，__tf.layers.dense__提供了很多其他的能力，包括设置多个正则参数。为了简单，在这个例子中接受默认的其他参数。

In [ ]:
# Build the hidden layers, sized according to the 'hidden_units' param.
for units in param['hidden_units']:
    net = tf.layers.dense(net, units=units, activation=tf.n.relu)

#### Output Layer
通过调用__tf.layers.dense__定义输出层，但是不用激活函数，示例如下：

这里，__net_表明最后一层hidden layer。因此，这个网络的连接如下图所示：
![Title](../image/add_logits.png)

当定义一个输出层的时候，__units__参数指定输出的个数。所以，通关过设定__units__到__patams['n_classes']，这个模型为每类产生一个输出。输出的每个单元都会包含一个评分(score)或者"logit",为Iris对应的相关类计算Setosa, Versicolor, or Virginica。

然后，这些logits会通过__tf.n.softmax__函数转换成概率。

In [ ]:
# Compute logits (1 per class).
logits = tf.layers.dense(net,params['n_classes'], activation=None)

### Implement training, evaluation and preodiction
创建模型函数的最后一步是实现prediction，evaluationh和training的分支代码。

当有人调用Estimators的train, evaluation或predict方法，模型函数会被唤醒。掉哦那个这个模型函数的签名如下代码所示：
> def my_model_fn(  
    features, # This is batch_features from input_fn  
    labels,   # This is batch_labels from input_fn  
    mode,     # An instance of tf.estimator.ModeKeys, see below  
    params):  # Additional configuration

专注于第三个参数,mode。在下面的表格中显示，当有人调用train,evaluate和predict，Estimator框架让你的模型函数的mode参数的设定如下：

Estimator method	|Estimator Mode
:-:|:-:
train()	|ModeKeys.TRAIN
evaluate()	|ModeKeys.EVAL
predict()	|ModeKeys.PREDICT

例如，假设你实例化你的自定义Estimator来产生一个对象命名为__classifier__。然后，你做如下的调用：
Estimator框架将mode设置为__ModeKeys.TRAIN__调用你的模型函数。

你的模型函数必须要提供能够处理所有的mode值得code。对每个mode value，code必须要返回一个tf.estimator.EstimatorSpec得实例，这个包含着调用者要求得信息。

In [ ]:
classifier = tf.estimator.Estimator(...)
classifier.train(input_fn=lambda:my_input_fn(FILE_TRAIN,True,500))

#### Predict
当调用Estimator得predict方法得时候，__model_fn__接受__mode=ModeKyes.PREDICT_。在这种情况下，模型函数必须要返回一包含prediction的tf.estimator.EstimatorSpec。

这个model必须要被train之后才能够做prediction。这个训练的模型被储存在磁盘被在实例化Estimator创建的__model_dir__目录上。

这个model生成prediction的code如下：

这个prediction字典包含着你模型运行在prediction mode。
![Title](../image/add_predictions.png)

__predictions__ 包含下面三个key/value对：
* __class_ids__ 包含着class id(0,1,or2)代表模型预测的这个例子中最可能的种类。
* __probabilities__ 有三个概率(在这个例子中， 0.02, 0.95, and 0.03)
* __logit__ 有原始的logit值(在这个例子中，-1.3, 2.6, and -0.9)

调用会通过__tf.estimator.EstimatorSpec__的__predictions__参数返回一个字典。这个Estimator的__predict__方法会生成这些字典。

In [ ]:
# Compute predictions.
predicted_classes = tf.argmax(logits,1)
if mode==tf.estimator.ModeKys.PREDICT:
    predictions={
        'class_ids': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits),
        'logits': logits,
    }
    return tf.estimator.EstimatorSpec(mode,predictions=predictions)

#### Calculate the loss
在training和evaluation都需要计算模型的loss。这就是优化的目标。

调用__tf.losses.sparse_sotfmax_cross_entropy__可以计算loss。 这个函数返回的值会很小，大约阶级0，正确类的概率大约接近1。当正确类的概率降低的时候，loss的返回值会逐渐增大。

这函数会返回这个batch的平均值

In [ ]:
# Compute loss
loss = tf.losses.sparse_sotfmax_cross_entropy(labels=labels,logits=logits)

#### Evaluate
当调用Estimator的evaluate方法的时候，__model_fn__接收__mode=ModeKeys.EVAL__。在这种情况下，模型函数返回一个包含模型loss和选择的一种或者多种的metrics的__tf.estimator.EstimatorSpec。
真实值
虽然返回metrics是选择项，但是大部分的自定义的Estimator会返回至少一个metrics。TensorFlow提供一个Metrics模块__tf.metrics__来计算正常的metrics。为了简洁起见，我们只返回精度。__tf.metric.accuracy__函数将预测值与真实值比较，这些真实值来自己输入函数的提供。这个函数要求预测值和真实值有相同的shape。下面是调用__tf.metrics.accuracy__示例如1:

evaluation返回的__EstimatorSpec__一般包含下面的信息：
* __loss__ ， 模型的loss
* _eval_metric_ops__ , 可选择的metric字典

因此，我们创造一个字典包含单一的metric。如果需要计算另外的metrics, 需要添加额外的key/value对到同一个字典。然后，将这个字典传递到__tf.estimator.EstimatorSpec__的__eval_metric_ops__参数中，示例如2：

__tf.sumary.scalar__ 在TRAIN和EVAL mode下让accuracy提供给 Tensoboard

In [ ]:
# 1:
# Compute evaluation metircs:
accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op'
                              )
# 2:
metrics = {'accuracy':accuracy}
tf.summary.scalar('accuracy',accuracy[1])

if mode==tf.estimator.ModeKyes.EVAL:
    return f.estimator.EstimatorSpec(mode,loss=loss,eval_metric_ops=metrics)

#### Trai
当Estimator调用train方法，__model_fn__调用__mode=ModeKeys.TRAIN。在这个情况下，模型函数返回一个含有loss和training operation的EstimatorSpec。

建立一个training operation会要求一个optimizer。 由于是模仿DNNClassifier，其中也是用Adagrad，我们会使用__tf.train.AdagradOptimizer__。__tf.train__包提供了很多其他的optimizer。

下面的代码建立了一个optimizer。

建立完optimizer之后，通过使用optimizer在loss上面的minimize方法创建training operation。

这个minimize方法也会添加__global_step__参数。TensorFLow使用这个参数来计算training steps已经运行过的次数(可以知道怎么样结束训练的次数)。__global_step__是TensorBoard graphs正常工作的必要参数。 简单调用__tf.train.get_global_step__并且将结果传达到__minimize__的__global_step__。下面的代码中详见怎么样训练模型：

training返回的EstimatorSpec有余下字段设置(field set)：
* __loss__ ， 包含loss函数值
* __train_op__ , 执行一个training step.
详见代码

In [ ]:
optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)

train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

### The custom Estimator
自定义Estimator的实例化通过Estimator的基础类，代码如下1所示：

这个示例代码中的__params__中的字典和DNNClassifier中的关键参数具有同样的目的。也就是说，__params__字典让你可以不修改__model_fn__而配置Estimator。

定义完Estimator之后，其中train，evaluate，predictions和之前一样。如下代码2所示：

In [ ]:
# 1:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    }
)

# 2:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, args.batch_size),
    steps=args.train_steps)

### TensorBoard
可以在TensorBoard中查看training result
> \# Replace PATH with the actual path passed as model_dir  
tensorboard --logdir=PATH

通过浏览器网址[http://localhost:6006/](http://localhost:6006/) 打开TensorBoard。（最好使用Google浏览器）

所有的pre-made Estimator都会自动载入很多信息到TensorBoard。在自定义的Estimator，虽然，TensorBoard只会提供一个默认的日志(a graph of the loss)外加你显示的告诉TensorBoard应该记录的信息。对于你创建的自定义的Estimator，TensorBoard长生一下图片：
![Title](../image/accuracy.png)
![Title](../image/loss.png)
![Title](../image/steps_per_second.png)

简单来说，这三个图告诉你：
* global_step/sec: 这个表现知识说明模型训练的时候，美妙有多少batches(梯度升级)
* loss: 表明loss
* accuracy： accuracy由两条线记录：
    * eval_metric_ops={'my_accuracy': accuracy},在evaluation的时候
    * tf.summary.scalar('accuracy', accuracy[1])，在训练的时候。

这些tensorboard的图是传递__global_step__到optimizer的__minimize__方法的主要原因之一。这个模型如果没有这个参数就无法记录x坐标。

注意到下面的这些__my_accuracy__和__loss__图:
* 黄色的线代表训练时候的结果
* 蓝色的点代表evaluation的结果。

在训练的时候，summaries(黄色的线)会随着batches的处理周期的记录，这也是为什么这个图会成为跨越x轴的图形。

与之不同，evaluation只会在每次调用evaluate的时候产生一个点。这个点包含了当整个evaluation调用的时候的平均值。在一个特定的训练步骤（从单一的checkpoint）,由于是评估整个模型的状态，所以这个图没有任何的宽度。

如下图所示，可以通过左边的可选圆圈来选择TensorBoard需要显示的曲线。
![Title](../image/select_run.jpg)

## Summary

虽然pre-made Estimators可以快速的创建一个模型，但是custom Estimators提供更多的领命多。幸运的是，pre-made和custom Estimator使用的是同一个编程模型。在实际当中不同的就是在custom Estimator时，必须要自己协定模型函数。
* The [official TensorFlow implementation of MNIST](https://github.com/tensorflow/models/tree/master/official/mnist), which uses a custom estimator.
* The [TensorFlow official models repository](https://github.com/tensorflow/models/tree/master/official), which contains more curated examples using custom estimators.
* This [TensorBoard video](https://ipv6.google.com/sorry/index?continue=https://youtu.be/eBbEDRsCmv4&q=EhAgAQJQMABL8xm08wdmAu0NGJb3v9MFIhkA8aeDS5SM5bnJ8QGbBcPEMpMT9BcmXiStMgFy), which introduces TensorBoard.
* The [Low Level Introduction](https://www.tensorflow.org/programmers_guide/low_level_intro?hl=zh-cn), which demonstrates how to experiment directly with TensorFlow's low level APIs, making debugging easier.

## Creating Estimators from Keras models

你可以将现在的Keras models转换到Estimator。 这样就要让Keras模型由进入Estimator的能力，例如分散训练。 调用__tf.keras.estimator.model_to_estimator__如下例所示：

注意到上例当中的fature columns的名字和keras estimator的labels都来自与相应的已经编译好的keras模型。例如，__train_input_fn__中输入key的名字可以通过__keras_inception_v3.input_names__来获得，同样的，预测的输出的名字可以通过__keras_inception_v3.output_names__。

详细的信息查看 [tf.keras.estimator.model_to_estimator](https://www.tensorflow.org/api_docs/python/tf/keras/estimator/model_to_estimator?hl=zh-cn)

In [ ]:
# Instantiate a Keras inception v3 model.
keras_inception_v3 = tf.keras.applications.inception_v3.InceptionV3(weights=None)
# Compile model with the optimizer, loss, and metrics you'd like to train with.
keras_inception_v3.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9),
                          loss='categorical_crossentropy',
                          metric='accuracy')
# Create an Estimator from the compiled Keras model. Note the initial model
# state of the keras model is preserved in the created Estimator.
est_inception_v3 = tf.keras.estimator.model_to_estimator(keras_model=keras_inception_v3)

# Treat the derived Estimator as you would with any other Estimator.
# First, recover the input name(s) of Keras model, so we can use them as the
# feature column name(s) of the Estimator input function:
keras_inception_v3.input_names  # print out: ['input_1']
# Once we have the input name(s), we can create the input function, for example,
# for input(s) in the format of numpy ndarray:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"input_1": train_data},
    y=train_labels,
    num_epochs=1,
    shuffle=False)
# To train, we call Estimator's train function:
est_inception_v3.train(input_fn=train_input_fn, steps=2000)